In [122]:
import pyspark
from pyspark.sql import SparkSession , Row , Window
from pyspark.sql.functions import * 
from pyspark.sql.functions import sum as spark_sum
from pyspark.sql.types import *
import sys , os
!{sys.executable} -m pip install beautifulsoup4

In [2]:
os.environ["PYSPARK_PYTHON"] = sys.executable
os.environ["PYSPARK_DRIVER_PYTHON"] = sys.executable
os.environ["HADOOP_HOME"] = "C:/Spark/spark-3.5.5-bin-hadoop3"  
os.environ["HADOOP_OPTS"] = "-Djava.library.path=C:/Spark/spark-3.5.5-bin-hadoop3/bin"

In [123]:
spark = SparkSession.builder.appName("PlatinumLayer_Transformations")\
    .config("spark.executor.memory", "5g")\
    .config("spark.jars.packages", "com.databricks:spark-xml_2.12:0.16.0,org.apache.parquet:parquet-hadoop:1.15.1")\
    .config("spark.pyspark.python", sys.executable) \
    .config("spark.pyspark.driver.python", sys.executable) \
    .config("spark.hadoop.io.native.lib.available", "false")\
    .getOrCreate()

## Extraction

In [124]:
GoldDataSet_path = "GoldDataSet"

# Read parquet files from each folder
answers_desc_dim_df = spark.read.parquet(f"{GoldDataSet_path}/Answers_Desc_Dim")
badge_desc_dim_df = spark.read.parquet(f"{GoldDataSet_path}/Badge_Desc_Dim")
badge_fact_df = spark.read.parquet(f"{GoldDataSet_path}/Badge_Fact")
comment_fact_df = spark.read.parquet(f"{GoldDataSet_path}/Comment_Fact")
date_dim_df = spark.read.parquet(f"{GoldDataSet_path}/Date_Dim")
qs_tags_bridge_df = spark.read.parquet(f"{GoldDataSet_path}/Qs_Tags_Bridge")
question_desc_dim_df = spark.read.parquet(f"{GoldDataSet_path}/Question_Desc_Dim")
questions_fact_df = spark.read.parquet(f"{GoldDataSet_path}/Questions_Fact")
tags_dim_df = spark.read.parquet(f"{GoldDataSet_path}/Tags_Dim")
users_dim_df = spark.read.parquet(f"{GoldDataSet_path}/Users_Dim")
vote_types_dim_df = spark.read.parquet(f"{GoldDataSet_path}/vote_types_dim")
votes_fact_df = spark.read.parquet(f"{GoldDataSet_path}/Votes_fact")

In [125]:
Answer_Fact_df = spark.read.parquet(f"{GoldDataSet_path}/Answer_Fact")


In [126]:
Answer_Silver_df = spark.read.parquet(f"SilverDataSet/Answers")

In [ ]:
Answer_Silver_df.show()     ###

+--------+--------+-----------+------------+----------------+--------------------+-----+------------+
|AnswerId|ParentId|OwnerUserId|CreationDate|LastActivityDate|                Body|Score|CommentCount|
+--------+--------+-----------+------------+----------------+--------------------+-----+------------+
|       5|       3|         23|  2010-07-19|      2010-07-19|the rproject http...|   90|           3|
|       9|       3|         50|  2010-07-19|      2010-07-19|incanter is a clo...|   15|           3|
|      12|       7|          5|  2010-07-19|      2010-07-19|see my response t...|   24|           1|
|      13|       6|         23|  2010-07-19|      2010-07-19|machine learning ...|   27|           6|
|      14|       3|         36|  2010-07-19|      2010-07-19|i second that jay...|    6|           1|
|      15|       1|          6|  2010-07-19|      2010-07-19|john cook gives s...|   24|           0|
|      16|       3|          8|  2010-07-19|      2010-07-19|two projects spri...|

In [ ]:
Answer_Fact_df.show()       ###

+------------------+--------------+---------------+---------------+-----------------+-----+------------+------------+----------------+
|Answer_Desc_Dim_FK|OwnerUserId_FK|CreationDate_FK|LastActivity_FK|ParentQuestion_FK|Score|CommentCount|CreationDate|LastActivityDate|
+------------------+--------------+---------------+---------------+-----------------+-----+------------+------------+----------------+
|              1937|          1342|       20100917|       20100917|              370|    4|           1|  2010-09-17|      2010-09-17|
|              5383|          3794|       20110319|       20110319|             2306|    0|           0|  2011-03-19|      2011-03-19|
|              5384|          3794|       20110319|       20110319|             2312|    2|           2|  2011-03-19|      2011-03-19|
|              5394|          3794|       20110320|       20110320|             2283|    1|           5|  2011-03-20|      2011-03-20|
|              5396|          3794|       20110320|    

In [ ]:
answers_desc_dim_df.show()      ###

+--------------+--------------+-----------+--------+--------------------+
|Answer_Desc_SK|Answer_Desc_BK|OwnerUserId|ParentId|                Body|
+--------------+--------------+-----------+--------+--------------------+
|            21|            49|          5|      33|you dont need to ...|
|           101|           174|        159|     170|online books incl...|
|           230|           357|       3807|       2|you cant know whe...|
|           286|           433|          5|     423|heres another one...|
|           337|           495|        214|     485|there is one call...|
|           470|           678|         81|     665|the probability o...|
|           877|          1247|        601|    1241|the shortest answ...|
|          1275|          1819|        561|    1815|if youre interest...|
|          1443|          2051|        930|    1908|artificial intell...|
|          1878|          2710|        187|    2688|you could use gen...|
|         42086|        100184|       

In [ ]:
comment_fact_df.show()      ###

+--------------+----------------+---------------+-------+-----+--------------------+------------+---------------+
|Answer_Desc_FK|Question_Desc_FK|CreationDate_FK|User_FK|Score|                Text|CreationDate|LastUpdatedDate|
+--------------+----------------+---------------+-------+-----+--------------------+------------+---------------+
|            -1|               5|       20100719|     87|    1|Very subjective q...|  2010-07-19|     2010-07-19|
|            41|              -1|       20100719|     44|    1|I wasn't suggesti...|  2010-07-19|     2010-07-19|
|            65|              -1|       20100720|     50|    0|http://cscs.umich...|  2010-07-20|     2010-07-20|
|            -1|              84|       20100720|     -1|   11|If you would tast...|  2010-07-20|     2010-07-20|
|            -1|              84|       20100720|      5|    0|IMO, this should ...|  2010-07-20|     2010-07-20|
|            -1|              84|       20100720|     -1|    0|@Sharpie It is no...|  20

**reading posts from bronze layer**

In [127]:
posts_schema=StructType([
    StructField("AcceptedAnswerId", IntegerType(), True),
    StructField("AnswerCount",IntegerType(), True),
    StructField("Body", StringType(), True),
    StructField("ClosedDate", TimestampType(), True),
    StructField("CommentCount",IntegerType(), True),
    StructField("CommunityOwnedDate", TimestampType(), True),
    StructField("ContentLicense", StringType(), True),
    StructField("CreationDate", TimestampType(), True),
    StructField("FavoriteCount",IntegerType(), True),
    StructField("Id",IntegerType(), False),
    StructField("LastActivityDate", TimestampType(), True),
    StructField("LastEditDate", TimestampType(), True),
    StructField("LastEditorDisplayName", StringType(), True),
    StructField("LastEditorUserId",IntegerType(), True),
    StructField("OwnerDisplayName", StringType(), True),
    StructField("OwnerUserId",IntegerType(), True),
    StructField("ParentId",IntegerType(), True),
    StructField("PostTypeId",IntegerType(), True),
    StructField("Score",IntegerType(), True),
    StructField("Tags", StringType(), True),
    StructField("Title", StringType(), True),
    StructField("ViewCount",IntegerType(), True)
])

df_posts = spark.read.format("xml") \
    .option("rowTag", "row")\
    .option("attributePrefix", "") \
    .schema(posts_schema) \
    .load(r"Dataset/Posts.xml")  

In [ ]:
badge_desc_dim_df.show()        ###

+-------------+--------------------+------+
|Badge_Desc_Id|                Name| Class|
+-------------+--------------------+------+
|            1|             p-value|Bronze|
|            2|             Marshal|  Gold|
|            3|               anova|Silver|
|            4|     autocorrelation|Bronze|
|            5|    Stellar Question|  Gold|
|            6|             Curious|Bronze|
|            7|   predictive-models|Bronze|
|            8|             moments|Bronze|
|            9|          self-study|Bronze|
|           10|       Good Question|Silver|
|           11|           Outspoken|Silver|
|           12|central-limit-the...|Bronze|
|           13|        optimization|Bronze|
|           14|generalized-linea...|Silver|
|           15|computational-sta...|Bronze|
|           16|      standard-error|Bronze|
|           17|         probability|Bronze|
|           18|          covariance|Bronze|
|           19|  distance-functions|Bronze|
|           20|              sam

## Programming Languages Analysis

In [105]:
language_patterns = {
    "python": r"python|Python",
    "java": r"java|Java",
    "javascript": r"javascript|JavaScript|js|JS",
    "ruby": r"ruby|Ruby",
    "php": r"php|PHP",
    "swift": r"swift|Swift",
    "kotlin": r"kotlin|Kotlin",
    "scala": r"scala|Scala",
    "typescript": r"typescript|TypeScript|ts|TS",
    "rust": r"rust|Rust",
    "dart": r"dart|Dart",
    "shell": r"shell|Shell|bash|Bash",
    "powershell": r"powershell|PowerShell|ps|PS",
    "perl": r"perl|Perl",
    "haskell": r"haskell|Haskell",
    "matlab": r"matlab|MATLAB",
    "objective-c": r"objective[- ]c|Objective[- ]C|objc|ObjC",
    "sql": r"\bsql\b|SQL",
    "html": r"html|HTML",
    "css": r"css|CSS",
    "c++": r"c\+\+|C\+\+",
    "c#": r"c#|C#|c sharp|C Sharp"
}

# Create month column
posts_plat_prog_langs = df_posts.withColumn("month", date_format("CreationDate", "yyyy-MM"))

# HTML-aware detection
for lang, pattern in language_patterns.items():
    # Special handling for C++ and C# in HTML
    if lang in ["c++", "c#"]:
        # Match both plain text and HTML entities (C++ might be encoded as C&amp;#43;&amp;#43;)
        html_encoded_pattern = pattern.replace("+", "&#43;").replace("#", "#")
        regex_pattern = f"(?i)({pattern})|({html_encoded_pattern})"
    else:
        regex_pattern = f"(?i)\\b({pattern})\\b"
    
    posts_plat_prog_langs = posts_plat_prog_langs.withColumn(
        lang,
        when(
            (col("body").rlike(regex_pattern)) | 
            (col("body").contains(f">{pattern}<")),  # Check within HTML tags
            1
        ).otherwise(0)
    )

# Rest of the processing remains the same
aggregations = [sum(lang).alias(f"{lang}_count") for lang in language_patterns.keys()]
monthly_counts_df = posts_plat_prog_langs.groupBy("month").agg(*aggregations).orderBy("month")

# Unpivot and calculate cumulative counts
num_languages = len(language_patterns)
stack_expr = "stack({0}, {1}) as (language, count)".format(
    num_languages,
    ", ".join([f"'{lang}', `{lang}_count`" for lang in language_patterns.keys()])
)
unpivoted_df = monthly_counts_df.select("month", expr(stack_expr))

window_spec = Window.partitionBy("language").orderBy("month").rowsBetween(Window.unboundedPreceding, 0)
final_Programming_langs_df2 = unpivoted_df.withColumn("overall_count", sum("count").over(window_spec))

final_Programming_langs_df2.select("month", "language", "count", "overall_count").orderBy("month")

DataFrame[month: string, language: string, count: bigint, overall_count: bigint]

In [106]:
final_Programming_langs_df2.show()

+-------+--------+-----+-------------+
|  month|language|count|overall_count|
+-------+--------+-----+-------------+
|2009-02|      c#|    0|            0|
|2009-04|      c#|    0|            0|
|2009-06|      c#|    0|            0|
|2009-12|      c#|    0|            0|
|2010-02|      c#|    0|            0|
|2010-07|      c#|    2|            2|
|2010-08|      c#|    0|            2|
|2010-09|      c#|    0|            2|
|2010-10|      c#|    1|            3|
|2010-11|      c#|    1|            4|
|2010-12|      c#|    5|            9|
|2011-01|      c#|    0|            9|
|2011-02|      c#|    1|           10|
|2011-03|      c#|    2|           12|
|2011-04|      c#|    1|           13|
|2011-05|      c#|    1|           14|
|2011-06|      c#|    0|           14|
|2011-07|      c#|    1|           15|
|2011-08|      c#|    2|           17|
|2011-09|      c#|    3|           20|
+-------+--------+-----+-------------+
only showing top 20 rows



In [ ]:
final_Programming_langs_df2.where(col('month')== "2020-10").show(30)        ###

+-------+-----------+-----+-------------+
|  month|   language|count|overall_count|
+-------+-----------+-----+-------------+
|2020-10|         c#|    6|          258|
|2020-10|        c++|    2|          405|
|2020-10|        css|    1|           89|
|2020-10|       dart|    1|           51|
|2020-10|    haskell|    0|           19|
|2020-10|       html|  147|        14567|
|2020-10|       java|    2|          638|
|2020-10| javascript|    2|          271|
|2020-10|     kotlin|    0|            2|
|2020-10|     matlab|   16|         3557|
|2020-10|objective-c|    0|            1|
|2020-10|       perl|    0|           68|
|2020-10|        php|    7|         1473|
|2020-10| powershell|   16|         1623|
|2020-10|     python|   80|         6187|
|2020-10|       ruby|    0|           54|
|2020-10|       rust|    0|           10|
|2020-10|      scala|    1|           59|
|2020-10|      shell|    1|          100|
|2020-10|        sql|    4|          238|
|2020-10|      swift|    0|       

In [117]:
final_Programming_langs_df2.coalesce(1).write.mode("overwrite").parquet("PlatinumDataSet/Programming_Languages_analysis")

## Data Engineering Tools analysis

In [ ]:
data_engineering_tools_patterns = {
    "ssis": r"ssis|SSIS|SQL Server Integration Services",
    "ssas": r"ssas|SSAS|SQL Server Analysis Services",
    "ssms": r"ssms|SSMS|SQL Server Management Studio",
    "apache_spark": r"apache spark|\bspark\b|Spark",
    "apache_kafka": r"apache kafka|\bkafka\b|Kafka",
    "apache_airflow": r"apache airflow|\bairflow\b|Airflow",
    "dbt": r"\bdbt\b|DBT|data build tool",
    "snowflake": r"\bsnowflake\b|Snowflake",
    "docker": r"\bdocker\b|Docker",
    "kubernetes": r"\bkubernetes\b|Kubernetes|\bk8s\b",
    "terraform": r"\bterraform\b|Terraform",
    "tableau": r"\btableau\b|Tableau",
    "fivetran": r"\bfivetran\b|Fivetran",
    "apache_flink": r"apache flink|\bflink\b|Flink",
    "apache_hadoop": r"apache hadoop|\bhadoop\b|Hadoop",
    "apache_hive": r"apache hive|\bhive\b|Hive",
    "apache_kudu": r"apache kudu|\bkudu\b|Kudu",
    "apache_cassandra": r"apache cassandra|\bcassandra\b|Cassandra",
    "cloudera": r"\bcloudera\b|Cloudera",
    "bigquery": r"bigquery|big query|BigQuery|Big Query",
    "looker": r"\blooker\b|Looker",
    "redash": r"\bredash\b|Redash",
    "presto": r"\bpresto\b|Presto",
    "power_bi": r"power bi|powerbi|Power BI|PowerBI",
    "aws_glue": r"aws glue|\bglue\b|AWS Glue|Glue",
    "azure_data_factory": r"azure data factory|Azure Data Factory|ADF",
    "meltano": r"\bmeltano\b|Meltano",
    "airbyte": r"\bairbyte\b|Airbyte",
    "databricks": r"\bdatabricks\b|Databricks",
    "amazon_redshift": r"amazon redshift|\bredshift\b|Redshift|Amazon Redshift",
    "postgresql": r"\bpostgresql\b|PostgreSQL|\bpostgres\b|Postgres",
    "mongodb": r"\bmongodb\b|MongoDB",
    "knime": r"\bknime\b|KNIME",
    "talend": r"\btalend\b|Talend",
    "prefect": r"\bprefect\b|Prefect",
    "streamlit": r"\bstreamlit\b|Streamlit",
    "kibana": r"\bkibana\b|Kibana",
    "debezium": r"\bdebezium\b|Debezium",
    "rivery": r"\brivery\b|Rivery",
    "cloudera_data": r"cloudera data|Cloudera Data",
    "arcticdb": r"\barcticdb\b|ArcticDB",
    "qdrant": r"\bqdrant\b|Qdrant",
    "snowpark": r"\bsnowpark\b|Snowpark",
    "anthropic": r"\banthropic\b|Anthropic",
    "bloomberg": r"\bbloomberg\b|Bloomberg",
    "sap": r"\bsap\b|SAP",
    "palantir": r"\bpalantir\b|Palantir",
    "salesforce": r"\bsalesforce\b|Salesforce",
    "replit": r"\breplit\b|Replit",
    "codeium": r"\bcodeium\b|Codeium",
    "sourcegraph": r"\bsourcegraph\b|Sourcegraph",
    "wired": r"\bwired\b|Wired",
    "simplilearn": r"\bsimplilearn\b|Simplilearn",
    "datacamp": r"\bdatacamp\b|DataCamp",
    "projectpro": r"\bprojectpro\b|ProjectPro",
    "binariks": r"\bbinariks\b|Binariks",
    "infomineo": r"\binfomineo\b|Infomineo",
    "ccs_learning_academy": r"\bccs learning academy\b|CCS Learning Academy",
    "redpanda": r"\bredpanda\b|Redpanda",
    "secoda": r"\bsecoda\b|Secoda"
}

# Create month column
posts_plat_tools = df_posts.withColumn("month", date_format("CreationDate", "yyyy-MM"))

# HTML-aware detection for tools
for tool, pattern in data_engineering_tools_patterns.items():
    # Handle special characters in tool names
    if any(char in tool for char in ['+', '#', '-', '_']):
        # Create HTML-encoded versions
        html_encoded_pattern = (pattern
                              .replace('+', '&#43;')
                              .replace('#', '&#35;')
                              .replace('-', '&#45;')
                              .replace('_', '&#95;'))
        regex_pattern = f"(?i)({pattern})|({html_encoded_pattern})"
    else:
        regex_pattern = f"(?i)\\b({pattern})\\b"
    
    posts_plat_tools = posts_plat_tools.withColumn(
        tool,
        when(
            (col("body").rlike(regex_pattern)) | 
            (col("body").contains(f">{pattern.split('|')[0]}<")),  # Check within HTML tags
            1
        ).otherwise(0)
    )

# Group by month and sum the binary indicators for each tool
aggregations = [sum(tool).alias(f"{tool}_count") for tool in data_engineering_tools_patterns.keys()]
monthly_counts_df = posts_plat_tools.groupBy("month").agg(*aggregations).orderBy("month")

# Unpivot the wide table to a long format: (month, tool, count)
num_tools = len(data_engineering_tools_patterns)
stack_expr = "stack({0}, {1}) as (tool, count)".format(
    num_tools,
    ", ".join([f"'{tool}', `{tool}_count`" for tool in data_engineering_tools_patterns.keys()])
)
unpivoted_df = monthly_counts_df.select("month", expr(stack_expr))

# Define window specification for cumulative counts
window_spec = Window.partitionBy("tool").orderBy("month").rowsBetween(Window.unboundedPreceding, 0)
final_data_eng_tools_df2 = unpivoted_df.withColumn("overall_count", sum("count").over(window_spec))

# Final output
final_data_eng_tools_df2.select("month", "tool", "count", "overall_count").orderBy("month")

DataFrame[month: string, tool: string, count: bigint, overall_count: bigint]

In [ ]:
final_data_eng_tools_df2.cache()

DataFrame[month: string, tool: string, count: bigint, overall_count: bigint]

In [115]:
final_data_eng_tools_df2.where(col('month')=='2020-11').show(30)

+-------+------------------+-----+-------------+
|  month|              tool|count|overall_count|
+-------+------------------+-----+-------------+
|2020-11|               dbt|    0|            0|
|2020-11|azure_data_factory|    7|          927|
|2020-11|        kubernetes|    0|            0|
|2020-11|              ssis|    0|            0|
|2020-11|  apache_cassandra|    0|            6|
|2020-11|          bigquery|    0|            9|
|2020-11|            presto|    0|           11|
|2020-11|        postgresql|    0|           19|
|2020-11|        projectpro|    0|            0|
|2020-11|            replit|    0|            0|
|2020-11|            secoda|    0|            2|
|2020-11|           prefect|    0|            8|
|2020-11|           codeium|    0|            0|
|2020-11|        salesforce|    0|           12|
|2020-11|             knime|    0|           29|
|2020-11|              ssas|    0|            3|
|2020-11|           mongodb|    0|            7|
|2020-11|           

In [118]:
final_data_eng_tools_df2.coalesce(1).write.mode("overwrite").parquet("PlatinumDataSet/DataEng_Tools_analysis")

In [ ]:
aggregated_df2 = final_data_eng_tools_df2.groupBy('tool').agg({'count': 'sum'})

# Then, order by 'overall_count' in descending order
aggregated_df2.orderBy(desc('sum(count)')).show()                       ###

+------------------+----------+
|              tool|sum(count)|
+------------------+----------+
|       apache_hive|      1619|
|azure_data_factory|      1201|
|      apache_spark|       279|
|     apache_hadoop|        79|
|           tableau|        75|
|          aws_glue|        72|
|          datacamp|        62|
|             wired|        57|
|             knime|        31|
|               sap|        25|
|        postgresql|        23|
|            presto|        17|
|         bloomberg|        16|
|   amazon_redshift|        16|
|      apache_flink|        15|
|              ssms|        14|
|          power_bi|        13|
|          bigquery|        12|
|        salesforce|        12|
|           prefect|        11|
+------------------+----------+
only showing top 20 rows



## Tags Over Time Analysis

In [128]:

tags_dim_df.createOrReplaceTempView("tags")
qs_tags_bridge_df.createOrReplaceTempView("qtb")
questions_fact_df.createOrReplaceTempView("question")
date_dim_df.createOrReplaceTempView("date")
Tags_Over_Time_df = spark.sql("""
WITH daily_counts AS (
    SELECT
        d.Month,
        d.Year,
        t.tagname,
        d.FullDate,
        COUNT(*) AS daily_count
    FROM tags t
    JOIN qtb q ON t.tag_sk = q.TagID
    JOIN question qn ON q.Questions_Desc_SK = qn.Questions_Desc_FK
    JOIN date d ON qn.CreationDate_FK = d.DateKey
    GROUP BY d.Month, d.Year, t.tagname, d.FullDate
),
monthly_counts AS (
    SELECT
        Month,
        Year,
        tagname,
        SUM(daily_count) AS monthly_count
    FROM daily_counts
    GROUP BY Month, Year, tagname
),
with_cumulative AS (
    SELECT
        mc.Month,
        mc.Year,
        mc.tagname,
        mc.monthly_count,
        SUM(mc.monthly_count) OVER (
            PARTITION BY mc.tagname
            ORDER BY mc.Year, mc.Month
            ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
        ) AS monthly_cumulative_count
    FROM monthly_counts mc
)
SELECT
    Month,
    Year,
    tagname,
    monthly_count,
    monthly_cumulative_count
FROM with_cumulative
ORDER BY tagname, Year, Month
""")


In [129]:
Tags_Over_Time_df.show()


+-----+----+--------------+-------------+------------------------+
|Month|Year|       tagname|monthly_count|monthly_cumulative_count|
+-----+----+--------------+-------------+------------------------+
|   10|2010|absolute-value|            2|                       2|
|   11|2010|absolute-value|            1|                       3|
|    1|2011|absolute-value|            2|                       5|
|    2|2011|absolute-value|            1|                       6|
|    3|2011|absolute-value|            4|                      10|
|    5|2011|absolute-value|            1|                      11|
|    8|2011|absolute-value|            3|                      14|
|    9|2011|absolute-value|            2|                      16|
|   10|2011|absolute-value|            2|                      18|
|   11|2011|absolute-value|            3|                      21|
|   12|2011|absolute-value|            2|                      23|
|    1|2012|absolute-value|            3|                     

In [133]:
Tags_Over_Time_df.coalesce(1).write.parquet("PlatinumDataSet/Tags_Over_Time")


In [134]:
spark.stop()